<a href="https://colab.research.google.com/github/ykitaguchi77/Colab_Scripts/blob/master/Whisper%20on%20google%20colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Whisper on google colab**

https://qiita.com/walnut-pro/items/0124a5a0c83c9b4e2669

https://qiita.com/walnut-pro/items/69864b0a074bd773711f

In [ ]:
! pip install git+https://github.com/openai/whisper.git --q

In [2]:
import whisper
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 77.6MiB/s]


In [3]:
# confirm device 
model.device #device(type='cuda', index=0)

device(type='cuda', index=0)

# **保存用フォルダ作成**

In [5]:
import os

# Add directory into content folder
checkDownLoadFolder = os.path.exists("download")
if not checkDownLoadFolder:
  os.mkdir("download")

# **YouTubeから音声をDL**

In [4]:
!pip install youtube-dl --q
!pip install yt-dlp --q
!pip install ffmpeg --q
#!python -m pip install -U yt-dlp

#from __future__ import unicode_literals
import youtube_dl
import pandas as pd
from yt_dlp import YoutubeDL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 41.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [7]:
ydl_opts = {
    'format': 'best',
    'outtmpl': f"001.mp4",  # name the location
}



with YoutubeDL(ydl_opts) as ydl:
    url_list = ["https://www.youtube.com/watch?v=N3vAAynw0G0"] #リストを直接入力する場合
    #url_list = df.iloc[:, 0] #CSV
    ydl.download(url_list)


[youtube] Extracting URL: https://www.youtube.com/watch?v=N3vAAynw0G0
[youtube] N3vAAynw0G0: Downloading webpage
[youtube] N3vAAynw0G0: Downloading android player API JSON
[info] N3vAAynw0G0: Downloading 1 format(s): 22
[download] Destination: 001.mp4
[download] 100% of  123.33MiB in 00:00:02 at 48.80MiB/s  


# **文字起こし**

In [8]:
fileName = "001.mp4"

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio(f"/content/{fileName}")

outputTextsArr = []
while audio.size > 0:
  tirmedAudio = whisper.pad_or_trim(audio)
  # trimedArray.append(tirmedAudio)
  startIdx = tirmedAudio.size
  audio = audio[startIdx:]

  # make log-Mel spectrogram and move to the same device as the model
  mel = whisper.log_mel_spectrogram(tirmedAudio).to(model.device)

  # detect the spoken language
  _, probs = model.detect_language(mel)
  # print(f"Detected language: {max(probs, key=probs.get)}")

  # decode the audio
  options = whisper.DecodingOptions()
  result = whisper.decode(model, mel, options)

  # print the recognized text
  outputTextsArr.append(result.text)

outputTexts = ' '.join(outputTextsArr)
print(outputTexts)

# Write into a text file
with open(f"download/{fileName}.txt", "w") as f:
  f.write(f"▼ Transcription of {fileName}\n")
  f.write(outputTexts)

ミルクの大学連合中ですさ今回は男女の連合感の違い5センということでね男女では実はね連合感連合の考え方そして連合の本能ですねこれが全く違うわけですよでね皆さんもうめちゃめちゃこれ超重要なんで覚えておいてくださいこれ本当に実感できればもう持ておうとごましくるんです女性に困ることはなくなります女性から寄ってくるようなめちゃめちゃかっこよーとこになれる まぎゃくです。これがね、これが僕は理解するのにね、ほんとね、死亡人かかりましたけども、男女ってシステムがまぎゃくなんですよ。生物構造上ね。だってさ、男がさ、出てる側でさ、女の子が、こう、へこんでる側じゃないですか。なんとなく分かる。で、その構造を元に、我々の本のっていうのは作られているわけですよ。 こうやってはうれしいって言うのと女の子はこうやってはうれしいって言うのが全然違うんですよということですじゃあよじゃあよこれさこれさ知らないとできなくないそうなんですよ知らないとね勉強しないとできないんですよじゃあ女の子はどうやったら好きになるのかどうやったら相性を感じるのかとかねどんなプレゼントを私たら喜ぶのかとかどんな関係してるのぞんでるんですかどうすれば上気にするんですか とかねみんな気になるよねそうなんですよけどねこれさ我々の感覚でいやキットこうやってら女の子って喜んでくれるに違いないと思ってやるでしょもうまた外れも話しそして女性からはこうもあれますあこいつわかってねえなとね悲しいよねせっかく女の子喜ばせだと思ってね幸せにしようと思ってやってることは知識不足によってもう水の淡ですすべてか水の淡は残念でしたってなるわけですなので今回はですね ポイント いつつバッチリお伝えします これ全部見て理解していただけたら女性を喜ばせる方法 女性を好きにさせる方法そして好きやった女性とずっといい関係を続けていこう方法 なんだるね読めさになってから結婚しからもずっといい関係を 気づいていこう方だったりとかもちろん連外関係だけじゃなくて 会社の中の女性とか女友だちとか同じクラス同じサクルですっていう女の子からもこの人分かっているなとこの人もてるな この人是非だなって思われて いいことばっかりですのでぜひ全部押さえてそして女心がわかる最強の持てをとこになっていきましょうでは行ってみましょうさあでは順番に行きましょう1つ目講義の最大知さあいつが一